In [1]:
import os
import glob
import boto3
import json
import uuid
import boto3
from strands import Agent
from strands.models import BedrockModel

# Try importing Strands
try:
    from strands import Agent
    from strands.models import BedrockModel
    STRANDS_AVAILABLE = True
except ImportError:
    STRANDS_AVAILABLE = False

print(f"Strands Available: {STRANDS_AVAILABLE}")

ModuleNotFoundError: No module named 'boto3'

In [2]:
# Configuration: True(Strands)/False(Bedrock) 선택, Strands 없으면 자동으로 Bedrock Agents 모드 적용
USE_STRANDS = False

print(f"Using: {'Strands (Local)' if USE_STRANDS else 'Bedrock Agents (Managed)'}")

Using: Bedrock Agents (Managed)


In [3]:
# Load available papers
papers = glob.glob("../data/papers/*.pdf")
paper_titles = [os.path.basename(p) for p in papers]
print("Available Papers:")
for i, title in enumerate(paper_titles, 1):
    print(f"{i}. {title}")

Available Papers:
1. 2025_GenMol A Drug Discovery Generalist with Discrete Diffusion.pdf
2. 2024_bowen_li_Accelerating ionizable lipid discovery for mRNA delivery using machine learning and combinatorial chemistry.pdf
3. 2024_ NeurIPS-generative-model-for-synthesizing-ionizable-lipids-mtcs-approach.pdf
4. 2025-machine-learning-framework-to-predict-the-performance-of-lipid-nanoparticles-for-nucleic-acid.pdf
5. Generative AI for designing and validating easily synthesizable and structurally novel antibiotics.pdf
6. 2024_A Deep Generative Model for the Design of Synthesizable Ionizable Lipids.pdf


In [4]:
# Create specialized team with paper context
paper_context = """
Available research papers:
1. GenMol: A Drug Discovery Generalist with Discrete Diffusion (2025)
2. A Deep Generative Model for the Design of Synthesizable Ionizable Lipids (2024)
3. Generative AI for designing and validating easily synthesizable and structurally novel antibiotics (2024)
4. NeurIPS: Generative model for synthesizing ionizable lipids - MCTS approach (2024)
5. Accelerating ionizable lipid discovery for mRNA delivery using machine learning and combinatorial chemistry (2024)

Focus: AI-driven ionizable lipid design using Monte Carlo Search Tree and Discrete Diffusion approaches.
"""

In [5]:
# Model configuration (only used for Strands mode)
MODEL_ID = "us.anthropic.claude-sonnet-4-5-20250929-v1:0"
REGION = "us-west-2"

In [6]:
# Initialize agents based on available implementation
def initialize_agents_strands(model_id, paper_context):
    """Initialize agents using Strands"""
    model = BedrockModel(model_id=model_id, region_name=REGION)
    
    agents = {
        "MOGAM-Chem-Agent": Agent(name="Dr. MOGAM-Chem-Agent", model=model),
        "MOGAM-AI-Agent": Agent(name="Dr. MOGAM-AI-Agent", model=model),
        "MOGAM-LNP-Agent": Agent(name="Dr. MOGAM-LNP-Agent", model=model),
        "MOGAM-BI-Agent": Agent(name="Dr. MOGAM-BI-Agent", model=model),
        "MOGAM-Lead-Agent": Agent(name="Dr. MOGAM-Lead-Agent", model=model),
    }
    
    return agents

def initialize_agents_bedrock():
    """Initialize agents using AWS Bedrock Agents (managed service)"""
    bedrock_agent_runtime = boto3.client('bedrock-agent-runtime', region_name=REGION)
    
    AGENT_IDS = {
        'MOGAM-Chem-Agent': '0IPX1MMI2D',
        'MOGAM-AI-Agent': 'KQ9FJVLHQE',
        'MOGAM-LNP-Agent': 'Q8GN0FK7NV',
        'MOGAM-BI-Agent': 'XDPBOQN8YT',
        'MOGAM-Lead-Agent': 'RHUTNOTET1'
    }
    
    class BedrockAgent:
        def __init__(self, name, agent_id, runtime_client):
            self.name = name
            self.agent_id = agent_id
            self.runtime = runtime_client
            self.session_id = str(uuid.uuid4())
        
        def __call__(self, prompt):
            response = self.runtime.invoke_agent(
                agentId=self.agent_id,
                agentAliasId='TSTALIASID',
                sessionId=self.session_id,
                inputText=prompt
            )
            
            result = ""
            for event in response['completion']:
                if 'chunk' in event:
                    chunk = event['chunk']
                    if 'bytes' in chunk:
                        result += chunk['bytes'].decode('utf-8')
            
            return result
    
    agents = {
        name: BedrockAgent(name, agent_id, bedrock_agent_runtime)
        for name, agent_id in AGENT_IDS.items()
    }
    
    return agents

In [7]:
# Initialize agents based on configuration
if USE_STRANDS:
    print("Initializing Strands agents...")
    agents = initialize_agents_strands(MODEL_ID, paper_context)
else:
    print("Initializing Bedrock agents...")
    agents = initialize_agents_bedrock()

chemical_expert = agents["MOGAM-Chem-Agent"]
ai_expert = agents["MOGAM-AI-Agent"]
lnp_expert = agents["MOGAM-LNP-Agent"]
bioinfo_expert = agents["MOGAM-BI-Agent"]
team_lead = agents["MOGAM-Lead-Agent"]

print("✓ Agents initialized")

Initializing Bedrock agents...
✓ Agents initialized


In [8]:
meeting_topic = """
Meeting Objective: Design a comprehensive AI system for ionizable lipid discovery using:
1. Monte Carlo Search Tree (MCTS) approaches
2. Discrete Diffusion models

Key Questions:
- How can we combine MCTS and discrete diffusion for optimal lipid design?
- What are the critical design parameters for ionizable lipids?
- How to create ionizable lipid efficacy prediciton model.
- How do we ensure synthetic accessibility and novelty?
- What validation strategies should we implement?
"""

print("Meeting Topic:")
print(meeting_topic)

Meeting Topic:

Meeting Objective: Design a comprehensive AI system for ionizable lipid discovery using:
1. Monte Carlo Search Tree (MCTS) approaches
2. Discrete Diffusion models

Key Questions:
- How can we combine MCTS and discrete diffusion for optimal lipid design?
- What are the critical design parameters for ionizable lipids?
- How to create ionizable lipid efficacy prediciton model.
- How do we ensure synthetic accessibility and novelty?
- What validation strategies should we implement?



In [9]:
research_question = "We need to design efficient and novel ionzable lipid using AI. What are the key considerations?"

In [10]:
# Chemical Expert Input
chem_input = chemical_expert(
    f"From a chemical structure perspective: {research_question}"
)
# Convert to string if Strands AgentResult
chem_input = str(chem_input) if not isinstance(chem_input, str) else chem_input
print(f"Chemical Expert {chemical_expert.name}:")
print("\n" + "-" * 30)
print(chem_input)

Chemical Expert MOGAM-Chem-Agent:

------------------------------
From a chemical structure perspective, here are the key considerations for AI-driven ionizable lipid design:

## **Core Structural Requirements:**

1. **Ionizable Head Group (pKa ~6-7)**
   - Must be protonated at acidic pH for cellular uptake
   - Neutral at physiological pH to avoid toxicity
   - Tertiary amines are most common but explore quaternary and polyamines

2. **Hydrophobic Tail Design**
   - Branched vs. linear alkyl chains affect membrane interaction
   - Biodegradable linkages (esters, carbonates) for clearance
   - Optimal tail length: C8-C14 for balance of stability/degradability

3. **Linker Chemistry**
   - Ester bonds: fast biodegradation but potential premature breakdown
   - Amide/carbamate: more stable but slower clearance
   - Balance between stability and metabolic clearance

## **AI-Specific Design Considerations:**

1. **Synthetic Accessibility**
   - Design molecules with <5 synthetic steps
   

In [11]:
# AI Expert Input
ai_input = ai_expert(
    f"From an AI/ML perspective: {research_question}"
)
# Convert to string if Strands AgentResult
ai_input = str(ai_input) if not isinstance(ai_input, str) else ai_input
print(f"AI Expert {ai_expert.name}:")
print("\n" + "-" * 30)
print(ai_input)

AI Expert MOGAM-AI-Agent:

------------------------------
From an AI/ML perspective, here are the key considerations for designing efficient and novel ionizable lipids:

**1. Molecular Representation Strategy**
- Choose appropriate encoding schemes (SMILES, graphs, fingerprints)
- Consider discrete vs. continuous representations for generative models
- Balance between expressiveness and computational efficiency

**2. Generative Model Architecture**
- **Monte Carlo Tree Search (MCTS)**: Excellent for exploring chemical space systematically with guided search
  - Enables exploitation-exploration balance
  - Can incorporate domain knowledge through reward functions
  - Particularly effective for synthesizability constraints
  
- **Discrete Diffusion Models**: Powerful for learning complex distributions
  - Can generate diverse molecular structures
  - Better at capturing multi-modal distributions
  - Requires careful noise scheduling and training strategies

**3. Multi-Objective Optimizat

In [ ]:
# LNP Expert Input
lnp_input = lnp_expert(
    f"From an LNP formulation perspective: {research_question}"
)
# Convert to string if Strands AgentResult
lnp_input = str(lnp_input) if not isinstance(lnp_input, str) else lnp_input
print(f"LNP Expert {lnp_expert.name}:")
print("\n" + "-" * 30)
print(lnp_input)

In [ ]:
# Bioinformatics Expert Input
bi_input = bioinfo_expert(
    f"From a bioinformatics perspective: {research_question}"
)
# Convert to string if Strands AgentResult
bi_input = str(bi_input) if not isinstance(bi_input, str) else bi_input
print(f"Bioinformatics Expert {bioinfo_expert.name}:")
print("\n" + "-" * 30)
print(bi_input)

In [ ]:
# Team Lead synthesizes all expert inputs
synthesis_prompt = f"""
Based on the following expert inputs, provide a comprehensive project plan:

Chemical Expert Input:
{chem_input}

AI Expert Input:
{ai_input}

LNP Expert Input:
{lnp_input}

Bioinformatics Expert Input:
{bi_input}

Please synthesize these perspectives and provide:
1. Key priorities
2. Implementation roadmap
3. Resource requirements
4. Risk mitigation strategies
"""

lead_synthesis = team_lead(synthesis_prompt)
# Convert to string if Strands AgentResult
lead_synthesis = str(lead_synthesis) if not isinstance(lead_synthesis, str) else lead_synthesis
print(f"Team Lead {team_lead.name}:")
print("\n" + "=" * 60)
print(lead_synthesis)

## Summary

In [ ]:
print("\n" + "=" * 60)
print("MEETING SUMMARY")
print("=" * 60)
print(f"\nImplementation: {'Strands (Local)' if USE_STRANDS else 'Bedrock Agents (Managed)'}")
print(f"Model: {MODEL_ID}")
print(f"\nTopic: {meeting_topic}")
print(f"\nQuestion: {research_question}")
print(f"\nExperts Consulted: 5")
print(f"- Chemical Expert: {len(chem_input)} chars")
print(f"- AI Expert: {len(ai_input)} chars")
print(f"- LNP Expert: {len(lnp_input)} chars")
print(f"- Bioinformatics Expert: {len(bi_input)} chars")
print(f"- Team Lead Synthesis: {len(lead_synthesis)} chars")
print(f"\nTotal Response: {len(chem_input) + len(ai_input) + len(lnp_input) + len(bi_input) + len(lead_synthesis)} chars")